# Evaluate Model Performance on the Test Set 

In [1]:
import numpy as np
import tensorflow as tf

from data import load_file, process_data, create_data_loader, preProcessingIWSLT12

from transformers import BertTokenizer
from transformers import TFBertForMaskedLM

from model import create_model

from datetime import datetime
import os
import json

import sys

In [2]:
!ls Models/20200424_165143

checkpoint			 cp-011.ckpt.data-00000-of-00002
cp-002.ckpt.data-00000-of-00002  cp-011.ckpt.data-00001-of-00002
cp-002.ckpt.data-00001-of-00002  cp-011.ckpt.index
cp-002.ckpt.index		 cp-013.ckpt.data-00000-of-00002
cp-005.ckpt.data-00000-of-00002  cp-013.ckpt.data-00001-of-00002
cp-005.ckpt.data-00001-of-00002  cp-013.ckpt.index
cp-005.ckpt.index		 cp-014.ckpt.data-00000-of-00002
cp-008.ckpt.data-00000-of-00002  cp-014.ckpt.data-00001-of-00002
cp-008.ckpt.data-00001-of-00002  cp-014.ckpt.index
cp-008.ckpt.index		 hyperparameters.json


In [3]:
checkpoint_path = "Models/20200424_165143/cp-014.ckpt"

In [4]:
# punctuation_enc = {
#     'O': 0,
#     'PERIOD': 1,
# }

punctuation_enc = {
    'O': 0,
    'COMMA': 1,
    'PERIOD': 2,
    'QUESTION': 3
}

### Hyper-parameters

In [5]:
n = 100

vocab_size = 30522
segment_size = 32
batch_size = 5
train_layer_ind = -2  # 0 for all model, -2 for only top layer
num_epochs = 2

hyperparameters = {
    'vocab_size': vocab_size,
    'segment_size': segment_size,
    'batch_size': batch_size
}

In [6]:
# name of data with the sentences
data_name = "IWSLT12"
testSet_01 = 'Data' + data_name + '/extractTest_01.txt'

preProcessingIWSLT12(testSet_01)

data_test = load_file('./Data/testSet_02.txt')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

X_test, y_test = process_data(data_test, tokenizer, punctuation_enc, segment_size)
y_test = np.asarray(y_test)

In [7]:
# one hot encode the labels
y_test = tf.one_hot(y_test, 4, dtype='int64').numpy()

### Build the dataset

In [8]:
# extract_X = X_test[0:n]
# extract_y = y_test[0:n]
extract_X = X_test[0:]
extract_y = y_test[0:]

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((extract_X, extract_y))
dataset = dataset.batch(batch_size)

### Build the model

In [10]:
# build and compile model

bert_input = tf.keras.Input(shape=(segment_size), dtype='int32', name='bert_input')
x = TFBertForMaskedLM.from_pretrained('bert-base-uncased')(bert_input)[0]
x = tf.keras.layers.Reshape((segment_size*vocab_size,))(x)
dense_out = tf.keras.layers.Dense(4, activation='softmax')(x)

net = tf.keras.Model(bert_input, dense_out, name='network')

net.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[tf.keras.metrics.Recall(class_id=0, name='Rec_0'),
                       tf.keras.metrics.Precision(class_id=0, name='Prec_0'),
                       tf.keras.metrics.Recall(class_id=1, name='Rec_1'),
                       tf.keras.metrics.Precision(class_id=1, name='Prec_1'),
                       tf.keras.metrics.Recall(class_id=2, name='Rec_2'),
                       tf.keras.metrics.Precision(class_id=2, name='Prec_2'),
                       tf.keras.metrics.Recall(class_id=3, name='Rec_3'),
                       tf.keras.metrics.Precision(class_id=3, name='Prec_3'),
                      ])

In [11]:
# load the weights
net.load_weights(checkpoint_path)

### Evaluate the model

In [12]:
net.evaluate(dataset)

  42/1717 [..............................] - ETA: 9:47 - loss: 0.6474 - Rec_0: 1.0000 - Prec_0: 0.8341 - Rec_1: 0.0000e+00 - Prec_1: 0.0000e+00 - Rec_2: 0.0000e+00 - Prec_2: 0.0000e+00 - Rec_3: 0.0000e+00 - Prec_3: 0.0000e+00

KeyboardInterrupt: 